# Profile History

In [11]:
from dotenv import load_dotenv
import os

load_dotenv()
os.environ["LANGSMITH_PROJECT"]

'LANGCHAIN-BASIC'

In [12]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0.1,
    model='gpt-4.1-mini',
    verbose=True
)

In [13]:
from pydantic import BaseModel, Field, ValidationError
from langchain_core.output_parsers import PydanticOutputParser

# pydantic model
class Profile(BaseModel):
    name: str
    gender: str
    age: int
    
parser = PydanticOutputParser(pydantic_object=Profile)        
parser

PydanticOutputParser(pydantic_object=<class '__main__.Profile'>)

In [15]:
fmt = parser.get_format_instructions() # JSON 응답을 요청하는 프롬프트(예시 포함됨)
fmt

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "type": "string"}, "gender": {"title": "Gender", "type": "string"}, "age": {"title": "Age", "type": "integer"}}, "required": ["name", "gender", "age"]}\n```'

In [14]:
from typing import Dict 
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', 'JSON으로만 출력 \n\n 양식: {fmt}'),
    ('user', '고객 프롬프트에서 이름(name), 성별(gender), 나이(age)를 추출:\n\n{profile_info}')
]).partial(fmt=fmt) # invoke때 넣는게 아니라 미리 넣음

prompt

ChatPromptTemplate(input_variables=['profile_info'], input_types={}, partial_variables={'fmt': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "type": "string"}, "gender": {"title": "Gender", "type": "string"}, "age": {"title": "Age", "type": "integer"}}, "required": ["name", "gender", "age"]}\n```'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt'], input_types={}, partial_variables={}, template='JSON으로만 출력 \n\n 양식: {fmt}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_vari

In [17]:
chain = prompt | model | parser
chain

ChatPromptTemplate(input_variables=['profile_info'], input_types={}, partial_variables={'fmt': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "type": "string"}, "gender": {"title": "Gender", "type": "string"}, "age": {"title": "Age", "type": "integer"}}, "required": ["name", "gender", "age"]}\n```'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fmt'], input_types={}, partial_variables={}, template='JSON으로만 출력 \n\n 양식: {fmt}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_vari

In [19]:
profile_info = '제 이름은 전민우라고 하며 36세 남성입니다'

try:
    profile_json = chain.invoke({'profile_info': profile_info})
except ValidationError as e:
    print('pydantic error:', e)

print(profile_json)

name='전민우' gender='남성' age=36
